In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from pandas.io import sql
import snowflake.connector
import keyring
import psycopg2 
import time
from datetime import date, timedelta
from scipy import stats

pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_columns', 500)

from matplotlib import pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
sns.set_style("darkgrid")

In [2]:
snowflake_username = 'matthew.bessey@disneystreaming.com'

In [3]:
ctx = snowflake.connector.connect(authenticator='externalbrowser', 
                                  user=snowflake_username, 
                                  account='disneystreaming.us-east-1')

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...


In [4]:
# set date parameters for query

subscription_start_date_min = "'2019-11-12'"
subscription_start_date_max = "'2019-11-13'" # max of subscription start date
engagement_date = "'2019-12-19'" # date for which we want to pull engagement behaviors

In [5]:
query= """
select o.swid
, o.swid_holdout
, a.accountid
, s.subscription_id
, e.*
from subscription s
join account a on s.account_id = a.accountid
join oneid_combined o on a.swid = o.swid
join "DSS_PROD"."DISNEY_PLUS"."DIM_DISNEY_DAILY_ACCOUNT_ENGAGEMENT" e on a.accountid = e.account_id
where s.partner = 'disney'
and s.calculated_subscription_start_dtm >= {}
and s.CALCULATED_SUBSCRIPTION_START_DTM <= {}
--and s.is_entitled = 1
and e.ds = {}
and e.is_pre_launch != 1
limit 100000;
""".format(subscription_start_date_min, subscription_start_date_max, engagement_date)

In [6]:
# run the query and write to engagement
engagement = pd.read_sql(query,ctx)

In [12]:
# map columns to lowercase
engagement.columns = engagement.columns.str.lower()

In [13]:
# create function and apply for mapping of holdout groups on 'swid_holdout'
def holdout_grouping(df):
    if df['swid_holdout'] < 243:
        return "all marketing"
    elif df['swid_holdout'] >= 243 and df['swid_holdout'] < 246:
        return "no onboarding"
    else:
        return "no marketing"
    
engagement['marketing_holdout'] = engagement.apply(holdout_grouping,axis=1)

In [17]:
#drop rows w/ no entitlement data, rewrite as int
engagement = engagement.drop(engagement[engagement['is_entitled']=='unknown'].index,axis=0)
engagement.is_entitled = engagement.is_entitled.astype(int)

In [39]:
# remove unnecessary columns
engagementCleaned = engagement.drop(['swid', 'ds', 'is_flagged'
                                    ,'swid_holdout','accountid','subscription_id'
                                    ,'account_id'
                                    ,'last_account_subscription_signup_week'
                                    , 'first_account_subscription_signup_week'
                                    , 'is_pre_launch'
                                    , 'is_pre_launch_nltt'
                                    , 'subscription_state_upd'
                                    , 'last_stream_date'],axis=1)

In [37]:
# remove non-US countries and then country column
countryUS_filter = engagement['account_home_country'] == 'US'
engagementCleaned = engagementCleaned[countryUS_filter]
engagementCleaned = engagementCleaned.drop('account_home_country',axis=1)

In [38]:
# show columns
engagementCleaned.columns

Index(['first_account_subscription_signup_date',
       'last_account_subscription_signup_date', 'is_entitled',
       'is_entitled_l1', 'is_entitled_l7', 'is_entitled_l28',
       'is_entitled_itd', 'subscription_type', 'total_login_days_l1',
       'total_login_days_l7', 'total_login_days_l28', 'total_login_days_itd',
       'total_streams_l1', 'total_streams_l7', 'total_streams_l28',
       'total_streams_itd', 'num_streaming_profiles_l1',
       'num_streaming_profiles_l7', 'num_streaming_profiles_l28',
       'num_streaming_profiles_itd', 'num_general_streaming_profiles_l1',
       'num_general_streaming_profiles_l7',
       'num_general_streaming_profiles_l28',
       'num_general_streaming_profiles_itd', 'num_kids_streaming_profiles_l1',
       'num_kids_streaming_profiles_l7', 'num_kids_streaming_profiles_l28',
       'num_kids_streaming_profiles_itd', 'num_streaming_devices_l1',
       'num_streaming_devices_l7', 'num_streaming_devices_l28',
       'num_streaming_devices_itd',

In [ ]:
binaryConstructionList = ['']

In [11]:
# create active_binary_l1 with 1 = user was active in l1d
engagement['active_binary_l1'] = engagement.apply(lambda df:
                                                 1 if df['total_streams_l1'] > 0
                                                 else 0,
                                                 axis=1)

In [12]:
# create active_binary_l7 with 1 = user was active in l7d
engagement['active_binary_l7'] = engagement.apply(lambda df:
                                                 1 if df['total_streams_l7'] > 0
                                                 else 0,
                                                 axis=1)

In [59]:
engagement.groupby('marketing_holdout').mean()

,swid_holdout,is_flagged,is_entitled,is_pre_launch,is_pre_launch_nltt,total_login_days_l1,total_login_days_l7,total_login_days_l28,total_login_days_itd,total_streams_l1,total_streams_l7,total_streams_l28,total_streams_itd,num_streaming_profiles_l1,num_streaming_profiles_l7,num_streaming_profiles_l28,num_streaming_profiles_itd,num_general_streaming_profiles_l1,num_general_streaming_profiles_l7,num_general_streaming_profiles_l28,num_general_streaming_profiles_itd,num_kids_streaming_profiles_l1,num_kids_streaming_profiles_l7,num_kids_streaming_profiles_l28,num_kids_streaming_profiles_itd,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_devices_l28,num_streaming_devices_itd,account_total_stream_days_l1,account_total_stream_days_l7,account_total_stream_days_l28,account_total_stream_days_itd,account_profile_total_stream_days_l1,account_profile_total_stream_days_l7,account_profile_total_stream_days_l28,account_profile_total_stream_days_itd,total_stream_days_general_profiles_l1,total_stream_days_general_profiles_l7,total_stream_days_general_profiles_l28,total_stream_days_general_profiles_itd,total_stream_days_kids_profiles_l1,total_stream_days_kids_profiles_l7,total_stream_days_kids_profiles_l28,total_stream_days_kids_profiles_itd,total_stream_time_ms_l1,total_stream_time_ms_l7,total_stream_time_ms_l28,total_stream_time_ms_itd,total_stream_time_general_profiles_ms_l1,total_stream_time_general_profiles_ms_l7,total_stream_time_general_profiles_ms_l28,total_stream_time_general_profiles_ms_itd,total_stream_time_kids_profiles_ms_l1,total_stream_time_kids_profiles_ms_l7,total_stream_time_kids_profiles_ms_l28,total_stream_time_kids_profiles_ms_itd,total_stream_time_web_ms_l1,total_stream_time_web_ms_l7,total_stream_time_web_ms_l28,total_stream_time_web_ms_itd,total_stream_time_mobile_ms_l1,total_stream_time_mobile_ms_l7,total_stream_time_mobile_ms_l28,total_stream_time_mobile_ms_itd,total_stream_time_connected_tv_ms_l1,total_stream_time_connected_tv_ms_l7,total_stream_time_connected_tv_ms_l28,total_stream_time_connected_tv_ms_itd,total_stream_time_unknown_ms_l1,total_stream_time_unknown_ms_l7,total_stream_time_unknown_ms_l28,total_stream_time_unknown_ms_itd,days_since_last_stream,active_binary_l1,active_binary_l7
marketing_holdout,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
all marketing,120.929282,0.000005,0.905454,0.0,0.000425,0.697272,4.966922,20.352837,28.875093,2.425662,18.437428,82.990252,129.118066,0.584060,1.341176,1.773665,1.960009,0.551761,1.261964,1.657695,1.834168,0.032299,0.079868,0.121462,0.137333,0.688515,1.992831,3.401399,4.159825,0.470533,3.427558,14.355273,21.213637,0.584060,4.288620,18.159114,26.942947,0.551761,4.047511,17.138496,25.498710,0.032299,0.241110,1.020618,1.444237,4.059107e+06,3.171180e+07,1.450750e+08,2.269535e+08,3.841748e+06,3.003848e+07,1.378333e+08,2.166882e+08,217359.221527,1.673318e+06,7.241695e+06,1.026537e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.637660,0.470533,0.821288
no marketing,250.522198,0.000000,0.904419,0.0,0.000423,0.692680,4.942408,20.291970,28.794257,2.382849,18.250048,82.693949,128.693423,0.573499,1.327666,1.758664,1.944291,0.540173,1.247703,1.642660,1.818242,0.033325,0.080731,0.121321,0.137055,0.677843,1.971044,3.383541,4.146088,0.464132,3.399813,14.290279,21.128599,0.573499,4.238004,18.030007,26.776999,0.540173,3.991083,16.987610,25.302451,0.033325,0.246922,1.042396,1.474548,3.993232e+06,3.134367e+07,1.441103e+08,2.255594e+08,3.768841e+06,2.960643e+07,1.364990e+08,2.148186e+08,224391.599598,1.737237e+06,7.611268e+06,1.074080e+07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.718064,0.464132,0.818601
no onboarding,243.988559,0.000000,0.908470,0.0,0.000427,0.699368,4.966872,20.377946,28.910861,2.504483,18.818733,84.458504,131.010545,0.587688,1.353398,1.779670,1.963414,0.555968,1.273267,1.664447,1.838542,0.031720,0.080815,0.121158,0.136356,0.691556,2.002092,3.424821,4.174607,0.471226,3.443946,14.399

In [13]:
##(engagement.groupby('marketing_holdout').mean().iloc[0,]/engagement.groupby('marketing_holdout').mean().iloc[1,]-1)*100

In [16]:
engagement.groupby('marketing_holdout').median()

,swid_holdout,is_flagged,is_entitled,is_pre_launch,is_pre_launch_nltt,total_login_days_l1,total_login_days_l7,total_login_days_l28,total_login_days_itd,total_streams_l1,total_streams_l7,total_streams_l28,total_streams_itd,num_streaming_profiles_l1,num_streaming_profiles_l7,num_streaming_profiles_l28,num_streaming_profiles_itd,num_general_streaming_profiles_l1,num_general_streaming_profiles_l7,num_general_streaming_profiles_l28,num_general_streaming_profiles_itd,num_kids_streaming_profiles_l1,num_kids_streaming_profiles_l7,num_kids_streaming_profiles_l28,num_kids_streaming_profiles_itd,num_streaming_devices_l1,num_streaming_devices_l7,num_streaming_devices_l28,num_streaming_devices_itd,account_total_stream_days_l1,account_total_stream_days_l7,account_total_stream_days_l28,account_total_stream_days_itd,account_profile_total_stream_days_l1,account_profile_total_stream_days_l7,account_profile_total_stream_days_l28,account_profile_total_stream_days_itd,total_stream_days_general_profiles_l1,total_stream_days_general_profiles_l7,total_stream_days_general_profiles_l28,total_stream_days_general_profiles_itd,total_stream_days_kids_profiles_l1,total_stream_days_kids_profiles_l7,total_stream_days_kids_profiles_l28,total_stream_days_kids_profiles_itd,total_stream_time_ms_l1,total_stream_time_ms_l7,total_stream_time_ms_l28,total_stream_time_ms_itd,total_stream_time_general_profiles_ms_l1,total_stream_time_general_profiles_ms_l7,total_stream_time_general_profiles_ms_l28,total_stream_time_general_profiles_ms_itd,total_stream_time_kids_profiles_ms_l1,total_stream_time_kids_profiles_ms_l7,total_stream_time_kids_profiles_ms_l28,total_stream_time_kids_profiles_ms_itd,total_stream_time_web_ms_l1,total_stream_time_web_ms_l7,total_stream_time_web_ms_l28,total_stream_time_web_ms_itd,total_stream_time_mobile_ms_l1,total_stream_time_mobile_ms_l7,total_stream_time_mobile_ms_l28,total_stream_time_mobile_ms_itd,total_stream_time_connected_tv_ms_l1,total_stream_time_connected_tv_ms_l7,total_stream_time_connected_tv_ms_l28,total_stream_time_connected_tv_ms_itd,total_stream_time_unknown_ms_l1,total_stream_time_unknown_ms_l7,total_stream_time_unknown_ms_l28,total_stream_time_unknown_ms_itd,days_since_last_stream,active_binary_l1,active_binary_l7
marketing_holdout,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
all marketing,121.0,0.0,1.0,0.0,0.0,1.0,6.0,24.0,33.0,0.0,8.0,47.0,84.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,3.0,15.0,22.0,0.0,4.0,16.0,24.0,0.0,3.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,16736881.0,93714004.0,159986427.0,0.0,15475739.0,87798517.0,151160071.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
no marketing,250.0,0.0,1.0,0.0,0.0,1.0,6.0,24.0,34.0,0.0,8.0,47.0,83.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,3.0,15.0,22.0,0.0,4.0,16.0,24.0,0.0,3.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,16588688.0,93034680.0,157585103.0,0.0,15309058.0,87018730.0,148728338.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
no onboarding,244.0,0.0,1.0,0.0,0.0,1.0,6.0,24.0,34.0,0.0,8.0,49.0,84.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,0.0,3.0,15.0,22.0,0.0,4.0,16.0,24.0,0.0,3.0,15.0,23.0,0.0,0.0,0.0,0.0,0.0,16986927.5,95372095.0,162220157.0,0.0,15863956.5,88671636.5,153850053.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [16]:

#q1 = stats.ttest_ind(engagement.loc[engagement['marketing_holdout']=='all marketing','is_entitled'],
 #                       engagement.loc[engagement.marketing_holdout=='no onboarding','is_entitled'])[1]

#print(q1)


In [30]:
#slice df for marketing and holdout slices

marketing_slice = engagement[engagement['marketing_holdout'] == 'all marketing']
holdout_slice = engagement[engagement['marketing_holdout'] == 'no marketing']

In [32]:
p_values = []

for i in range(16,len(marketing_slice.columns)):
    try: 
        p1 = stats.ttest_ind(marketing_slice.iloc[:,i],
                        holdout_slice.iloc[:,i])[1]
        descr1 = marketing_slice.columns[i]
    except (TypeError,RuntimeWarning): 
        p1 = "Broke!"
    p_values.append([descr1,p1])
    print("Completed",i)


Completed 16
Completed 17


/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


Completed 18
Completed 19
Completed 20
Completed 21
Completed 22
Completed 23
Completed 24
Completed 25
Completed 26
Completed 27
Completed 28
Completed 29
Completed 30
Completed 31
Completed 32
Completed 33
Completed 34
Completed 35
Completed 36
Completed 37
Completed 38
Completed 39
Completed 40
Completed 41
Completed 42
Completed 43
Completed 44
Completed 45
Completed 46
Completed 47
Completed 48
Completed 49
Completed 50
Completed 51
Completed 52
Completed 53
Completed 54
Completed 55
Completed 56
Completed 57
Completed 58
Completed 59
Completed 60
Completed 61
Completed 62
Completed 63
Completed 64
Completed 65
Completed 66
Completed 67
Completed 68
Completed 69
Completed 70
Completed 71
Completed 72
Completed 73
Completed 74
Completed 75
Completed 76
Completed 77
Completed 78
Completed 79
Completed 80
Completed 81
Completed 82
Completed 83
Completed 84
Completed 85
Completed 86
Completed 87
Completed 88
Completed 89
Completed 90
Completed 91


In [34]:
p_values

[['is_pre_launch', nan],
 ['is_pre_launch_nltt', 0.8493168085179987],
 ['is_pre_launch_nltt', 'Broke!'],
 ['is_pre_launch_nltt', 'Broke!'],
 ['is_pre_launch_nltt', 'Broke!'],
 ['total_login_days_l1', 0.12294656565361425],
 ['total_login_days_l7', 0.08797844880950126],
 ['total_login_days_l28', 0.019587342840840027],
 ['total_login_days_itd', 0.010880849218279927],
 ['total_streams_l1', 0.11661642143870479],
 ['total_streams_l7', 0.2660619922234634],
 ['total_streams_l28', 0.11724506396719173],
 ['total_streams_itd', 0.09099968452959808],
 ['num_streaming_profiles_l1', 1.9563725531440613e-05],
 ['num_streaming_profiles_l7', 0.004456732365602317],
 ['num_streaming_profiles_l28', 0.000279954709103938],
 ['num_streaming_profiles_itd', 0.00020447072107953083],
 ['num_general_streaming_profiles_l1', 2.91058212779101e-06],
 ['num_general_streaming_profiles_l7', 0.000868598332836167],
 ['num_general_streaming_profiles_l28', 0.00012502116423895935],
 ['num_general_streaming_profiles_itd', 0.000